In [1]:
import sys; sys.path.append("../")
import pandas as pd

from lib.explore import show_unique, show_columns_with_differences
from lib.split_schema import detect_unique_columns_subset
from clean.ipm_new_orleans_pd_cprr_allegations import clean
from clean.ipm_new_orleans_pd_cprr_actions_taken import clean as clean_at
from lib import events


In [2]:
complaints = clean()

/Users/khoipham/.virtualenvs/base/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (63,74) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


standardize_from_lookup_table: unmatched sequences:
  {'unknown race'}


In [3]:
complaints

,tracking_number,incident_type,officer_primary_key,allegation_primary_key,allegation,disposition,allegation_finding,allegation_class,occur_time,assigned_unit,...,allegation_create_year,allegation_create_month,allegation_create_day,occur_year,occur_month,occur_day,charges,data_production_year,agency,complaint_uid
0,1993-0549-R,rank initiated,2251,15567,paragraph 02 - instructions from authoritative...,sustained,sustained,rule 4: perf of duty,NaN,Un-assigned,...,2014,2,20,,,,rule 4: perf of duty; paragraph 02 - instructi...,2020,New Orleans PD,250d3eb480a0faf7dd617828ff016c32
1,1994-0132-R,rank initiated,770,14996,paragraph 06 - unauthorized force,not sustained,not sustained,rule 2: moral conduct,NaN,Un-assigned,...,2013,11,13,,,,,2020,New Orleans PD,2ca0081231c5c2fe04555df869ecc73e
2,1994-0238-C,public initiated,2251,15568,paragraph 01 - adherence to law,not sustained,not sustained,rule 2: moral conduct,NaN,Un-assigned,...,2014,2,20,,,,rule 2: moral conduct; paragraph 01 - adherenc...,2020,New Orleans PD,d1491bb478bbb53f884f3b0aae691256
3,1994-0294-R,rank initiated,2320,15529,paragraph 01 - adherence to law,sustained,sustained,rule 2: moral conduct,NaN,Un-assigned,...,2014,2,13,,,,rule 2: moral conduct; paragraph 01 - adherenc...,2020,New Orleans PD,959a5da0afcafb8f6eba20e804fb2272
4,1994-0531-C,public initiated,2251,15569,paragraph 04 - neglect of duty,not sustained,not sustained,rule 4: perf of duty,NaN,Un-assigned,...,2014,2,20,,,,rule 4: perf of duty; paragraph 04 - neglect o...,2020,New Orleans PD,53ab420da0944f6ca734541f6a676c63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23609,2019-0112-P,public initiated,882,26291,paragraph 04 - neglect of duty,pending,pending,rule 4: perf of duty,20:00,PIB Intake Section,...,2019,2,21,2018,8,14,rule 4: perf of duty; paragraph 04 - neglect o...,2020,New Orleans PD,21085554cca128e5d624844ccd8505f8
23610,2019-0112-P,public initiated,3719,26293,paragraph 04 - neglect of duty,pending,pending,rule 4: perf of duty,20:00,PIB Intake Section,...,2019,2,21,2018,8,14,rule 4: perf of duty; paragraph 04 - neglect o...,2020,New Orleans PD,446639a636a68ceff09b8e3dc62ed2a9
23611,2019-0112-P,public initiated,,,,pending,pending,unknown class,20:00,PIB Intake Section,...,,,,2018,8,14,rule 4: perf of duty; paragraph 04 - neglect o...,2020,New Orleans PD,542be7c1d3fccec18aca84559baae92d
23612,2019-0112-P,public initiated,5404,26292,paragraph 04 - neglect of duty,pending,pending,rule 4: perf of duty,20:00,PIB Intake Section,...,2019,2,21,2018,8,14,rule 4: perf of duty; paragraph 04 - neglect o...,2020,New Orleans PD,f7633db738644c5012a282726eb2b47d


In [9]:
complaints[~complaints.tracking_number.str.match(r'^\d{4}-\d+')]

,tracking_number,incident_type,officer_primary_key,allegation_primary_key,allegation,disposition,allegation_finding,allegation_class,occur_time,assigned_unit,...,allegation_create_year,allegation_create_month,allegation_create_day,occur_year,occur_month,occur_day,charges,data_production_year,agency,complaint_uid
7843,201-0146-D,rank initiated,474,11297,paragraph 04 - neglect of duty,illegitimate outcome,illegitimate outcome,rule 4: perf of duty,00:00,Field Operation Bureau,...,2012,4,13,2011,2,8,rule 4: perf of duty; paragraph 04 - neglect o...,2020,New Orleans PD,d3423b7658a1cd438541b0c2514d8ef4
7844,201-1328-D,rank initiated,3596,12098,paragraph 04 - neglect of duty,illegitimate outcome,illegitimate outcome,rule 4: perf of duty,10:00,Invest. Services Bureau,...,2012,8,20,2011,11,27,rule 4: perf of duty; paragraph 04 - neglect o...,2020,New Orleans PD,0ca69d9f19714f5aa4f369abe6eee3ff
23613,Feb-09,rank initiated,2064,12442,,pending,pending,unknown class,NaN,Management Serv. Bureau,...,2012,10,9,,,,,2020,New Orleans PD,dd53c6b9c9cdea07823de29f6f27461d
